In [191]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickl
import sys

from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [192]:
from sklearn.ensemble import RandomForestRegressor

In [239]:
df = pd.read_csv('C:/Users/dtulomba/Documents/DOSSIER_PROJET_AI/OneDrive_2023-09-14/Wind Turbine Component Temperature Prediction/df2.csv', index_col=0, parse_dates=True)

In [240]:
sub = pd.read_csv('C:/Users/dtulomba/Documents/DOSSIER_PROJET_AI/OneDrive_2023-09-14/Wind Turbine Component Temperature Prediction/submission_template.csv', parse_dates=True, index_col=0)

In [241]:
df

,ws:scada,wd:scada,wd:nacelle,pow:active,react,voltage:1,voltage:2,voltage:3,pow:current_1,pow:current_2,...,pres:0m,pres:sea,atmo:layer,cloud:cover,heat:surf,fric:velo,atmo:cloud,heat:sol,missing,dataset
datetime,,,,,,,,,,,,,,,,,,,,,
2016-05-03 09:40:00,4.728757,276.818054,268.365997,353.803345,-167.806335,403.092010,403.543335,403.127014,328.994995,341.654999,...,101022.833333,102470.466667,1032.400000,53.666667,-115.969333,0.412770,877.733333,525.193333,NaN,train
2016-05-03 09:50:00,6.427782,275.881958,268.365997,515.468567,-185.731003,403.238098,403.334290,403.119904,462.379486,469.053986,...,101027.966667,102475.033333,1041.500000,55.583333,-117.775667,0.410214,895.616667,520.171667,NaN,train
2016-05-03 10:00:00,5.876329,270.699310,268.365997,403.398895,-125.460991,403.584991,403.817596,403.573914,353.306000,363.613495,...,101033.100000,102479.600000,1050.600000,57.500000,-119.582000,0.407658,913.500000,515.150000,NaN,train
2016-05-03 10:10:00,5.676786,278.005402,268.365997,399.778290,-154.261169,402.217285,402.518311,402.269012,360.062500,366.775513,...,101034.683333,102480.816667,1062.416667,57.916667,-120.968500,0.407689,943.033333,530.891667,NaN,train
2016-05-03 10:20:00,4.777902,284.922363,268.365997,206.619904,-108.576736,402.087402,402.622711,402.163208,198.177505,206.764999,...,101036.266667,102482.033333,1074.233333,58.333333,-122.355000,0.407720,972.566667,546.633333,NaN,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-16 07:30:00,1.419412,109.917045,77.390907,-0.565510,0.000000,406.459454,408.065099,407.302905,5.117250,1.804250,...,100100.000000,101498.950000,410.200000,71.850000,-55.816500,0.126688,615.850000,282.925000,NaN,test
2020-06-16 07:40:00,1.763269,119.556042,77.390907,-0.324368,0.000000,406.150450,407.653453,406.833554,4.034250,1.213750,...,100100.033333,101498.166667,441.633333,71.233333,-60.040667,0.127937,571.900000,304.586667,NaN,test
2020-06-16 07:50:00,1.790783,126.096721,77.390907,-0.363342,0.000000,405.897315,407.605951,406.655375,4.418947,1.340000,...,100100.066667,101497.383333,473.066667,70.616667,-64.264833,0.129185,527.950000,326.248333,NaN,test


In [242]:
def score(y_true, y_pred, ret=False):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"RMSE: {rmse}")
    print(f"R2: {r2}")
    if ret:
        return rmse, r2

In [243]:
TARGET = 'gear_oil_temp'
IGNORE_COLS = ['missing', 'dataset', 'missing:target']

In [244]:
SCADA_FEATS = ['ws:scada', 'wd:scada', 'wd:yaw', 'wd:nacelle', 'pow:active', 'pow:apparent', 'pow:rotor', 'pow:current_1', 
               'pow:current_2', 'pow:current_3', 'react', 'bearing_temp:front', 'bearing_temp:back', 'gen_temp:rear', 
               'gen_temp:front', 'gen_temp:stator', 'int_temp:rotor', 'int_temp:nacelle', 'int_temp:top', 'ext_temp:amb', 
               'ext_temp:nacelle', 'ext_temp:hub', 'ext_temp:tcell', 'trans_temp', 'gen:speed', 'gen:gear', 
               'blade_pitch:a', 'blade_pitch:c', 'blade_pitch:b', 'voltage:1', 'voltage:2', 'voltage:3']
ERA_FEATS = ['ws:era_10m', 'ws:era_100m', 'wd:era_10m', 'wd:era_100m', 'ext_temp:era', 'humid:dew', 'humid:rh', 'pres:0m', 
             'pres:sea', 'atmo:layer', 'atmo:cloud', 'cloud:cover', 'heat:surf', 'heat:sol', 'fric:velo']

In [245]:
# Assurez-vous que df est défini dans votre environnement
# Si votre DataFrame est stocké dans un fichier CSV, vous pouvez le charger ainsi :
# df = pd.read_csv('votre_fichier.csv')

# Sélectionnez les données d'entraînement et de test
df_train = df.query('dataset == "train"').dropna(subset=[TARGET]).copy(deep=True)
df_test = df.query('dataset == "test"').dropna(subset=[TARGET]).copy(deep=True)

In [246]:
df_test.head(2)

,ws:scada,wd:scada,wd:nacelle,pow:active,react,voltage:1,voltage:2,voltage:3,pow:current_1,pow:current_2,...,pres:0m,pres:sea,atmo:layer,cloud:cover,heat:surf,fric:velo,atmo:cloud,heat:sol,missing,dataset
datetime,,,,,,,,,,,,,,,,,,,,,
2019-12-01 00:00:00,4.535865,45.804017,36.780682,206.646064,-83.449434,403.837149,404.2405,404.235899,186.873251,194.138501,...,100841.600000,102311.40,246.90,82.5,27.659000,0.20384,615.40,0.0,NaN,test
2019-12-01 00:10:00,5.208292,50.700098,36.780682,292.804819,-110.881642,403.670596,404.0312,404.130048,260.868750,268.805254,...,100839.466667,102309.35,244.15,80.1,26.175333,0.20663,614.15,0.0,NaN,test


In [247]:
df_train.head(2)

,ws:scada,wd:scada,wd:nacelle,pow:active,react,voltage:1,voltage:2,voltage:3,pow:current_1,pow:current_2,...,pres:0m,pres:sea,atmo:layer,cloud:cover,heat:surf,fric:velo,atmo:cloud,heat:sol,missing,dataset
datetime,,,,,,,,,,,,,,,,,,,,,
2016-05-03 09:40:00,4.728757,276.818054,268.365997,353.803345,-167.806335,403.092010,403.543335,403.127014,328.994995,341.654999,...,101022.833333,102470.466667,1032.4,53.666667,-115.969333,0.412770,877.733333,525.193333,NaN,train
2016-05-03 09:50:00,6.427782,275.881958,268.365997,515.468567,-185.731003,403.238098,403.334290,403.119904,462.379486,469.053986,...,101027.966667,102475.033333,1041.5,55.583333,-117.775667,0.410214,895.616667,520.171667,NaN,train


In [202]:
# Supprimez les lignes avec des valeurs manquantes dans les colonnes cibles
df_train = df_train.dropna(subset=[TARGET])
df_test = df_test.dropna(subset=[TARGET])

In [203]:
# Sélectionnez les caractéristiques (colonnes) pour l'entraînement
features = [c for c in df_train.columns if (c != TARGET) and (c not in IGNORE_COLS)]

# Séparez les caractéristiques et les étiquettes pour l'entraînement et le test
X_train, X_test = df_train[features].values, df_test[features].values
y_train, y_test = df_train[TARGET].values, df_test[TARGET].values

In [204]:
df_train.head(2)

,ws:scada,wd:scada,wd:nacelle,pow:active,react,voltage:1,voltage:2,voltage:3,pow:current_1,pow:current_2,...,pres:0m,pres:sea,atmo:layer,cloud:cover,heat:surf,fric:velo,atmo:cloud,heat:sol,missing,dataset
datetime,,,,,,,,,,,,,,,,,,,,,
2016-05-03 09:40:00,4.728757,276.818054,268.365997,353.803345,-167.806335,403.092010,403.543335,403.127014,328.994995,341.654999,...,101022.833333,102470.466667,1032.4,53.666667,-115.969333,0.412770,877.733333,525.193333,NaN,train
2016-05-03 09:50:00,6.427782,275.881958,268.365997,515.468567,-185.731003,403.238098,403.334290,403.119904,462.379486,469.053986,...,101027.966667,102475.033333,1041.5,55.583333,-117.775667,0.410214,895.616667,520.171667,NaN,train


In [206]:
# Faites des prédictions sur l'ensemble de test
y_preds = model.predict(X_test)

# Calculez l'erreur (vous devez définir votre propre fonction score)
scores = mean_squared_error(y_test, y_preds)  # Par exemple, utilisez MSE comme métrique

# Calculez l'erreur (vous devez définir votre propre fonction score)
mse = mean_squared_error(y_test, y_preds) # Par exemple, utilisez MSE comme métrique
rmse = mse ** 0.5

# Affichez les prédictions et les scores pour vérification
print("Prédictions sur l'ensemble de test :", y_preds)
print("MSE :", mse)
print("RMSE :", rmse)

Prédictions sur l'ensemble de test : [53.25674324 53.91402626 53.65946178 ... 37.80995308 37.3411281
 37.47270222]
MSE : 10.184248293374466
RMSE : 3.1912769064082274


In [207]:
# Convertissez votre tableau NumPy en une pandas Series
y_preds_series = pd.Series(y_preds)

In [208]:
submission_df_final2 = (
   y_preds_series
   .rename('1_Gear oil temperature (°C)')
   .reindex(sub.index)
   .fillna(y_preds_series.mean())
   .to_frame()
)

In [209]:
submission_df_final2.to_csv("submission_df_final2.csv")

### Rolling features

Rolling Features (Caractéristiques Glissantes) : Il s'agit d'un terme général qui englobe diverses caractéristiques calculées sur des fenêtres glissantes d'une série temporelle. Cela peut inclure des mesures telles que la moyenne, l'écart-type, le minimum, le maximum, etc.

Rolling Mean Difference (Différence Moyenne Glissante) : C'est une caractéristique spécifique qui calcule la différence entre la moyenne des valeurs dans une fenêtre glissante actuelle et la moyenne des valeurs dans une fenêtre glissante précédente.

### Rolling mean


In [248]:
roll_cols = ['ws:scada', 'pow:active', 'gen:gear', 'pow:apparent', 'ws:era_100m']
roll_cols += ['blade_pitch:a', 'gear_oil_temp', 'ext_temp:era']

In [249]:
for r in roll_cols:
    df[f"rolling_10:{r}"] = df[r].rolling(5).mean()
    df[f"rolling_20:{r}"] = df[r].rolling(15).mean()

### Longer period

### Rolling mean difference

In [250]:
# Remplacer les NaN par la dernière valeur non manquante (forward fill)
df.fillna(df.mean(), inplace=True)

C:\Users\dtulomba\AppData\Local\Temp/ipykernel_27476/3467461828.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True)


In [251]:
# Sélectionnez les données d'entraînement et de test
df_train = df.query('dataset == "train"').dropna(subset=[TARGET]).copy(deep=True)
df_test = df.query('dataset == "test"').dropna(subset=[TARGET]).copy(deep=True)

In [252]:
# Supprimez les lignes avec des valeurs manquantes dans les colonnes cibles
df_train = df_train.dropna(subset=[TARGET])
df_test = df_test.dropna(subset=[TARGET])

In [253]:
len(df_train)

182665

In [254]:
len(df_test)

28272

In [255]:
# Sélectionnez les caractéristiques (colonnes) pour l'entraînement
features = [c for c in df_train.columns if (c != TARGET) and (c not in IGNORE_COLS)]

# Séparez les caractéristiques et les étiquettes pour l'entraînement et le test
X_train, X_test = df_train[features].values, df_test[features].values
y_train, y_test = df_train[TARGET].values, df_test[TARGET].values

In [256]:
# Créez et entraînez le modèle
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [257]:
# Faites des prédictions sur l'ensemble de test
y_preds = model.predict(X_test)

# Calculez l'erreur (vous devez définir votre propre fonction score)
scores = mean_squared_error(y_test, y_preds)  # Par exemple, utilisez MSE comme métrique

mse = mean_squared_error(y_test, y_preds) # Par exemple, utilisez MSE comme métrique
rmse = mse ** 0.5

# Affichez les prédictions et les scores pour vérification
print("Prédictions sur l'ensemble de test :", y_preds)
print("MSE :", mse)
print("RMSE :", rmse)

Prédictions sur l'ensemble de test : [55.92355249 56.14014206 56.01401634 ... 37.60791607 37.38948684
 37.11678114]
MSE : 0.49694530249802066
RMSE : 0.7049434746829143


In [218]:
# Convertissez votre tableau NumPy en une pandas Series
y_preds_series = pd.Series(y_preds)

In [219]:
submission_df_final4 = (
   y_preds_series
   .rename('1_Gear oil temperature (°C)')
   .reindex(sub.index)
   .fillna(y_preds_series.mean())
   .to_frame()
)

In [220]:
submission_df_final4.to_csv("submission_df_final4.csv")

### XGBOOSTING

In [221]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [222]:
model_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

In [223]:
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [224]:
# Faites des prédictions sur l'ensemble de test
y_preds_xg = model_xgb.predict(X_test)

# Calculez l'erreur (vous devez définir votre propre fonction score)
scores = mean_squared_error(y_test, y_preds)  # Par exemple, utilisez MSE comme métrique

mse = mean_squared_error(y_test, y_preds) # Par exemple, utilisez MSE comme métrique
rmse = mse ** 0.5

# Affichez les prédictions et les scores pour vérification
print("Prédictions sur l'ensemble de test :", y_preds)
print("MSE :", mse)
print("RMSE :", rmse)

Prédictions sur l'ensemble de test : [55.84490505 56.22924839 55.93277493 ... 37.81685356 37.55680946
 37.3984605 ]
MSE : 0.511430869050265
RMSE : 0.7151439498802077


In [225]:
# Convertissez votre tableau NumPy en une pandas Series
y_preds_xg_boosting_series = pd.Series(y_preds_xg)

In [226]:
submission_df_xg_boosting_final = (
   y_preds_xg_boosting_series
   .rename('1_Gear oil temperature (°C)')
   .reindex(sub.index)
   .fillna(y_preds_xg_boosting_series.mean())
   .to_frame()
)

### Cat Boosting

In [227]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [228]:
model_catb = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, loss_function='RMSE', random_seed=42)

In [229]:
model_catb.fit(X_train, y_train)

0:	learn: 5.1315705	total: 30.2ms	remaining: 2.99s
1:	learn: 4.6765990	total: 53.1ms	remaining: 2.6s
2:	learn: 4.2558881	total: 73.7ms	remaining: 2.38s
3:	learn: 3.8739901	total: 97.1ms	remaining: 2.33s
4:	learn: 3.5368427	total: 120ms	remaining: 2.28s
5:	learn: 3.2346412	total: 142ms	remaining: 2.22s
6:	learn: 2.9612652	total: 164ms	remaining: 2.18s
7:	learn: 2.7179545	total: 189ms	remaining: 2.18s
8:	learn: 2.4984482	total: 215ms	remaining: 2.17s
9:	learn: 2.3029777	total: 244ms	remaining: 2.2s
10:	learn: 2.1293042	total: 265ms	remaining: 2.15s
11:	learn: 1.9700406	total: 291ms	remaining: 2.13s
12:	learn: 1.8342039	total: 311ms	remaining: 2.08s
13:	learn: 1.7100824	total: 332ms	remaining: 2.04s
14:	learn: 1.6038035	total: 354ms	remaining: 2.01s
15:	learn: 1.5081773	total: 377ms	remaining: 1.98s
16:	learn: 1.4230251	total: 402ms	remaining: 1.96s
17:	learn: 1.3502346	total: 425ms	remaining: 1.93s
18:	learn: 1.2853373	total: 452ms	remaining: 1.92s
19:	learn: 1.2288772	total: 475ms	remai

In [230]:
# Faites des prédictions sur l'ensemble de test
y_preds_catb = model_catb.predict(X_test)

# Calculez l'erreur (vous devez définir votre propre fonction score)
scores = mean_squared_error(y_test, y_preds)  # Par exemple, utilisez MSE comme métrique

mse = mean_squared_error(y_test, y_preds) # Par exemple, utilisez MSE comme métrique
rmse = mse ** 0.5

# Affichez les prédictions et les scores pour vérification
print("Prédictions sur l'ensemble de test :", y_preds)
print("MSE :", mse)
print("RMSE :", rmse)

Prédictions sur l'ensemble de test : [55.84490505 56.22924839 55.93277493 ... 37.81685356 37.55680946
 37.3984605 ]
MSE : 0.511430869050265
RMSE : 0.7151439498802077


In [231]:
# Convertissez votre tableau NumPy en une pandas Series
y_preds_cat_boosting_series = pd.Series(y_preds_catb)

In [232]:
submission_df_cat_boosting_final = (
   y_preds_cat_boosting_series
   .rename('1_Gear oil temperature (°C)')
   .reindex(sub.index)
   .fillna(y_preds_cat_boosting_series.mean())
   .to_frame()
)

In [233]:
submission_df_cat_boosting_final

,1_Gear oil temperature (°C)
# Date and time,
2020-06-16 08:20:00,52.351143
2020-06-16 08:30:00,52.351143
2020-06-16 08:40:00,52.351143
2020-06-16 08:50:00,52.351143
2020-06-16 09:00:00,52.351143
...,...
2021-06-16 07:40:00,52.351143
2021-06-16 07:50:00,52.351143
2021-06-16 08:00:00,52.351143


In [234]:
submission_df_cat_boosting_final.to_csv("submission_df_cat_boosting_final.csv")

### Liste pour montrer l'évolution de nos RMSE 